In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic #getting the holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utils to draw the lines

def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS) #FACEMESH_TESSELATION gives full face markings
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=2))

vid=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic: #set mediapipeline model
    while True:
        ret,frame=vid.read()

        image,results = mediapipe_detection(frame,holistic) #model detect
        #print(results)
        
        draw_styled_landmarks(image,results) #draw lines
        
        cv2.imshow("Frame",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    vid.release()
    cv2.destroyAllWindows()


plt.imshow(frame)

draw_landmarks(frame,results)

plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

def extract_keypoints(results):
    
    pose= np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    lh= np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    rh= np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    face= np.array([[res.x,res.y,res.z] for res in results.pose_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    
    return np.concatenate([pose,face,lh,rh])

DATA_PATH = os.path.join('MP_Data')
actions= np.array(['hello','thanks','iloveyou'])
no_sequences=30
sequence_length= 30 #frame length

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

vid=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic: #set mediapipeline model
    
    for action in actions:
        
        for sequence in range(no_sequence):
            
            for frame_num in range(sequence_length):
                
                ret,frame=vid.read()

                image,results = mediapipe_detection(frame,holistic) #model detect
                #print(results)

                draw_styled_landmarks(image,results) #draw lines
                
                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow("Frame",image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    
                    cv2.imshow("Frame",image)
                
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)


                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
    vid.release()
    cv2.destroyAllWindows()


from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map={label:num for num,label in enumerate(actions)}

sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(window)
        sequences.append(window)
        labels.append(label_map[action])

y= to_categorical(labels).astype(int)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir=os.path.join('Logs')
tb_callback=TensorBoard(log_dir=log_dir)

model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

model.fit(x_train,y_train,epochs=2000,callbacks=[tb_callback])

res=model.predict(x_test)

actions[np.argmax(res[0])]

actions[np.argmax(y_test[0])]

#model.save('action.h5') save the model

#model.load_weights('action.h5')

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat=model.predict(x_test)

ytrue=np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axxis=1).tolist()

multilabel_confusion_matrix(ytrue,yhat)

colors=[(245,117,16),(117,245,16),(16,117,245)]
def prob_viz(res,actions,input_frame,colors):
    output_frame=input_frame.copy()
    for num,prob in enumerate(res):
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

sequence=[]
sentence=[]
threshold=0.8

vid=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic: #set mediapipeline model
    while True:
        ret,frame=vid.read()

        image,results = mediapipe_detection(frame,holistic) #model detect
        #print(results)
        
        draw_styled_landmarks(image,results) #draw lines
        
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence=sequence[:30]
        
        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
        
        if res[np.argmax(res)]>threshold:
            if len(sentence)>0:
                if actions[np.argmax(res)]!= sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
            
        if len(sentence)>5: 
            sentence=sentence[-5:]
            
        image=prob_viz(res,actions,image,colors)
            
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE)
        
        cv2.imshow("Frame",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    vid.release()
    cv2.destroyAllWindows()


